# **IMPORT LIBS**

In [1]:
import numpy as np
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.recommenders import CML, BPR, PMF
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler
import numpy as np
import pandas as pd
import scipy.sparse as sps
import os
import pickle
import random


# **GENERATE THE DATASET**

In [2]:
seed = 2384795
np.random.seed(seed=seed)

folder_name = f"../Dataset/"

if os.path.exists(folder_name) == False:
    os.makedirs(folder_name)

## Load the dataset

In [3]:
file_path = 'yahoo_ymusic_v1/ydata-ymusic-rating-study-v1_0-train.txt'

# Load the training set into a DataFrame
df_train = pd.read_csv(folder_name+file_path, sep='\t',names=["UserID","SongID","Rating"], header=None)  # sep='\t' for tab-separated values
df_train.head(10)

,UserID,SongID,Rating
0,1,14,5
1,1,35,1
2,1,46,1
3,1,83,1
4,1,93,1
5,1,94,1
6,1,153,5
7,1,170,4
8,1,184,5
9,1,194,5


## Convert to implicit

"We treat items rated greater than or equal to 4 as relevant, and others as irrelevant, as suggested by prior literature."

In [4]:
POSITIVE_THRESHOLD = 4
df_train['ImplicitRating'] = np.where(df_train['Rating'] >= POSITIVE_THRESHOLD, 1, 0)

df_train.head(10)

,UserID,SongID,Rating,ImplicitRating
0,1,14,5,1
1,1,35,1,0
2,1,46,1,0
3,1,83,1,0
4,1,93,1,0
5,1,94,1,0
6,1,153,5,1
7,1,170,4,1
8,1,184,5,1
9,1,194,5,1


## Check the number of users and items in the training set

"The training set contains 300K ratings given by 15.4K users against 1K songs through natural interactions."

In [5]:
min_user = df_train["UserID"].min()
max_user = df_train["UserID"].max()

min_item = df_train["SongID"].min()
max_item = df_train["SongID"].max()

max_item, max_user

(1000, 15400)

# **GET UNBIASED TESTSET**

## Load the unbiased testset and convert it to implicit

In [6]:
file_path = '../Dataset/yahoo_ymusic_v1/ydata-ymusic-rating-study-v1_0-test.txt'
df_test = pd.read_csv(file_path, sep='\t',names=["UserID","SongID","Rating"], header=None)  # sep='\t' for tab-separated values
df_test['ImplicitRating'] = np.where(df_test['Rating'] >= POSITIVE_THRESHOLD, 1, 0)
df_test.head(10)

,UserID,SongID,Rating,ImplicitRating
0,1,49,1,0
1,1,126,1,0
2,1,138,1,0
3,1,141,1,0
4,1,177,1,0
5,1,268,3,0
6,1,511,1,0
7,1,587,1,0
8,1,772,5,1
9,1,941,1,0


## Check the number of users and items in the training set

"The testing set is collected by asking a subset of 5.4K users to rate 10 randomly selected songs."

In [7]:
df_test["UserID"].max(), df_test["SongID"].max(), int(df_test.shape[0]/df_test["UserID"].max())

(5400, 1000, 10)

## Filter unbiased testset

"We filter the testing set by retaining users who have at least a relevant and an irrelevant song in the testing set and two relevant songs in the training set."

In [8]:
# Select users with at least an irrelevant song in the unbiased testset
usersWithNegativeInteractionInTest = df_test[df_test["ImplicitRating"] == 0]["UserID"].unique()

# Select UserID of users with at least a relevant song in testset
usersWithPositiveInteractionInTest = df_test[df_test["ImplicitRating"] == 1]["UserID"].unique()

# Select UserID of users with at least two relevant song in trainset
usersWithTwoPositiveInteractions = df_train[df_train["ImplicitRating"] == 1].groupby("UserID").filter(lambda x: len(x) >= 2)['UserID'].unique()

# Compute the intersection
set1 = set(usersWithNegativeInteractionInTest)
set2 = set(usersWithPositiveInteractionInTest)
set3 = set(usersWithTwoPositiveInteractions)
valid_users_testset = set1 & set2 & set3

# Filter the testset
df_test_filtered = df_test[df_test["UserID"].isin(valid_users_testset)]

"2296 users satisfy these requirements."

In [9]:
len(valid_users_testset)

2296

## Shape the unbiased test set

In [10]:
# From the dataframe, for each row where ImplicitRating is 1, append [userID, itemID] to unbiased_pos_test_set
# and for each row where ImplicitRating is 0, append [userID, itemID] to unbiased_neg_test_set

unbiased_pos_test_set = df_test_filtered[df_test_filtered["ImplicitRating"] == 1][["UserID", "SongID"]].values
unbiased_neg_test_set = df_test_filtered[df_test_filtered["ImplicitRating"] == 0][["UserID", "SongID"]].values

## Save unbiased test set

In [11]:
unbiased_pos_test_set_df = pd.DataFrame(unbiased_pos_test_set)
unbiased_neg_test_set_df = pd.DataFrame(unbiased_neg_test_set)

unbiased_pos_test_set_df.columns = ["user_id","item_id"]
unbiased_neg_test_set_df.columns = ["user_id","item_id"]

structured_data_pos_test_set_unbiased = unbiased_pos_test_set_df.to_records(index=False)
structured_data_neg_test_set_unbiased = unbiased_neg_test_set_df.to_records(index=False)

np.save(folder_name + "unbiased-test_arr_pos.npy", structured_data_pos_test_set_unbiased)
np.save(folder_name + "unbiased-test_arr_neg.npy", structured_data_neg_test_set_unbiased)

# **GET BIASED TESTSET**

In [12]:
file_path = 'yahoo_ymusic_v1/ydata-ymusic-rating-study-v1_0-train.txt'

# Load the training set into a DataFrame
df_train = pd.read_csv(folder_name+file_path, sep='\t',names=["UserID","SongID","Rating"], header=None)  # sep='\t' for tab-separated values
df_train.head(10)

,UserID,SongID,Rating
0,1,14,5
1,1,35,1
2,1,46,1
3,1,83,1
4,1,93,1
5,1,94,1
6,1,153,5
7,1,170,4
8,1,184,5
9,1,194,5


In [13]:
df_train['ImplicitRating'] = np.where(df_train['Rating'] >= POSITIVE_THRESHOLD, 1, 0)


In [14]:
df_train = df_train[df_train["UserID"].isin(valid_users_testset)]

In [15]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 58799 entries, 0 to 129178
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   UserID          58799 non-null  int64
 1   SongID          58799 non-null  int64
 2   Rating          58799 non-null  int64
 3   ImplicitRating  58799 non-null  int64
dtypes: int64(4)
memory usage: 2.2 MB


## Extract the biased test set and shape it

"We additionally held out a biased testing set (biased-testing) from the training set by randomly sampling 300 songs for each user."

In [16]:
# Precompute, for each user, the list of songs with a relevant rating
user_positive_ratings = df_train[df_train["ImplicitRating"] == 1].groupby("UserID")["SongID"].apply(set)

# Initialize the range of indexes for the items
items_ids = np.arange(min_item, max_item + 1)
# Set the number of songs for each user
SONGS_FOR_BIASED_TEST = 300

#IPOTESI MAN

pos_test_set = []
neg_test_set = []

for user_id in valid_users_testset:
    np.random.shuffle(items_ids)
    test_items = set(items_ids[-SONGS_FOR_BIASED_TEST:])
    pos_ids = user_positive_ratings.get(user_id, set()) & test_items

    #set those to 0 so that they will no longer be used in training set
    df_train.loc[(df_train['SongID'].isin(pos_ids)) & (df_train['UserID'] == user_id), 'ImplicitRating'] = 0

    for id in test_items:
        if id in pos_ids:
            pos_test_set.append([user_id, id])
        else:
            neg_test_set.append([user_id, id])

pos_test_set = np.array(pos_test_set)
neg_test_set = np.array(neg_test_set)

## Save the biased test set

In [17]:
pos_test_set_df = pd.DataFrame(pos_test_set)
neg_test_set_df = pd.DataFrame(neg_test_set)

pos_test_set_df.columns = ["user_id","item_id"]
neg_test_set_df.columns = ["user_id","item_id"]

structured_data_pos_test_set = pos_test_set_df.to_records(index=False)
structured_data_neg_test_set = neg_test_set_df.to_records(index=False)

np.save(folder_name + "biased-test_arr_pos.npy", structured_data_pos_test_set)
np.save(folder_name + "biased-test_arr_neg.npy", structured_data_neg_test_set)

## **STORE TRAINSET**

## Take couples user-item filtering out the irrelevant ones

In [18]:
# Only take the couples (user, item) with relevant rating
new_df = df_train[df_train['ImplicitRating'] != 0]
new_df = new_df.drop(columns=['Rating', 'ImplicitRating'])

# Define a dictionary for renaming columns
rename_dict = {
    'UserID': 'user_id',
    'SongID': 'item_id'
}

# Rename the columns
new_df = new_df.rename(columns=rename_dict)

# Convert the DataFrame to a structured array
structured_data = new_df.to_records(index=False) 

In [19]:
train_data = structured_data

## Save the training set

In [20]:
np.save(folder_name + "training_arr.npy", train_data)

# **MODEL CHOICE**

In [21]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')

MODEL_CLASS = CML
MODEL_PREFIX = "cml"
DATASET_NAME = "yahoo"
OUTPUT_FOLDER = "./Output/"
OUTPUT_PATH = OUTPUT_FOLDER + MODEL_PREFIX + "-" + DATASET_NAME + "/"
OUTPUT_PREFIX = str(OUTPUT_PATH) + str(MODEL_PREFIX) + "-" + str(DATASET_NAME)


if os.path.exists(OUTPUT_PATH) == False:
    os.makedirs(OUTPUT_PATH)


# **TRAIN THE MODEL**

In [22]:
# Avoid tensorflow using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()


model = MODEL_CLASS(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(), 
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=model, sampler=sampler,
                                     eval_save_prefix=OUTPUT_PATH + DATASET_NAME,
                                     item_serving_size=500)
auc_evaluator = AUC()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






2024-03-29 12:36:16.773692: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2024-03-29 12:36:16.780314: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 4192029999 Hz
2024-03-29 12:36:16.781274: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556a42af2ea0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-03-29 12:36:16.781290: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [23]:
model_trainer.train(num_itr=10001, display_itr=display_itr)

== Start training with FULL evaluation ==
[Itr 100] Finished
[Itr 200] Finished
[Itr 300] Finished
[Itr 400] Finished
[Itr 500] Finished
[Itr 600] Finished
[Itr 700] Finished
[Itr 800] Finished
[Itr 900] Finished
[Itr 1000] Finished
INFO:tensorflow:./Output/cml-yahoo/yahoo-1000 is not in all_model_checkpoint_paths. Manually adding it.
[Itr 1000] loss: 1768.235659
[Itr 1100] Finished
[Itr 1200] Finished
[Itr 1300] Finished
[Itr 1400] Finished
[Itr 1500] Finished
[Itr 1600] Finished
[Itr 1700] Finished
[Itr 1800] Finished
[Itr 1900] Finished
[Itr 2000] Finished
INFO:tensorflow:./Output/cml-yahoo/yahoo-2000 is not in all_model_checkpoint_paths. Manually adding it.
[Itr 2000] loss: 649.306754
[Itr 2100] Finished
[Itr 2200] Finished
[Itr 2300] Finished
[Itr 2400] Finished
[Itr 2500] Finished
[Itr 2600] Finished
[Itr 2700] Finished
[Itr 2800] Finished
[Itr 2900] Finished
[Itr 3000] Finished
INFO:tensorflow:./Output/cml-yahoo/yahoo-3000 is not in all_model_checkpoint_paths. Manually adding it

In [24]:
model.save(OUTPUT_PATH,None)

INFO:tensorflow:./Output/cml-yahoo/ is not in all_model_checkpoint_paths. Manually adding it.


In [25]:
del model

# **DEFINING FUNCTIONS**

In [26]:
def eq(infilename, infilename_neg, trainfilename, gamma=-1.0, K=1):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset

    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0

    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)

            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            # Calcolo il Recall a 1, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
                

        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator 

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [27]:
def aoa(infilename, infilename_neg, trainfilename, K=1):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0

        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser]))
            # Calcolo il Recall a 30, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0
            denominator += 1 

        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [28]:
def stratified(infilename, infilename_neg, trainfilename, gamma=1.0, K=30, partition=10):

    # Read pickles
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]

    # Merge P and P_neg
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])

    Zui = dict()
    Ni = dict()

    # Compute frequencies of items in the training set
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset

    # Compute recommendations for each user
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))

    pui = dict()
    w = dict()

    # Compute dictionary of propensity scores
    for theuser in P["users"]:
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            if theitem in pui:
                continue
            pui[theitem] = np.power(Ni[theitem], (gamma + 1) / 2.0)

    # Take the list of items (not tuples) in pui sorted by value
    items_sorted_by_value = sorted(pui, key=pui.get, reverse=True)

    # Compute linspace between the pui[0] and pui[-1] 
    linspace = np.linspace(pui[items_sorted_by_value[0]], pui[items_sorted_by_value[-1]], partition+1)
   
    # Compute dictionary w, that is, for each item, assigns the average of the puis in the partition it belongs to
    i=0
    j = 0
    while i < len(items_sorted_by_value):
                            
        avg = 0
        start = i
        end = i
    
        while i < len(items_sorted_by_value) and pui[items_sorted_by_value[i]] >= linspace[j+1]:
            avg += 1.0 / pui[items_sorted_by_value[i]]
            end = i
            i += 1
        
        avg = avg / (end - start + 1)

        for k in range(start, end+1):
            w[items_sorted_by_value[k]] = avg


        j += 1

    nonzero_user_count = 0
    sum_user_auc = 0.0
    sum_user_recall = 0.0

    # Compute score with AUC and compute Recall
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            # Skip items with null frequency
            if  theitem not in Ni:
                continue
            # Add things to be summed for each item
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) * w[theitem]
            # Add things for recall
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 * w[theitem] # spetta
            # Increment denominator that the sum must be divided by 
            denominator += 1 / pui[theitem]


        # If there was at least one item for the user, count the user and sum the results
        if denominator > 0:
            nonzero_user_count += 1
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator 

    return {
        "auc"       : sum_user_auc / nonzero_user_count, 
        "recall"    : sum_user_recall / nonzero_user_count
    }

# **EVALUATION**

In [29]:
auc_results = []
recall_results = []

In [30]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['test_data_pos_biased'] = np.load(folder_name + "biased-test_arr_pos.npy")
raw_data['test_data_neg_biased'] = np.load(folder_name + "biased-test_arr_neg.npy")
raw_data['test_data_pos_unbiased'] = np.load(folder_name + "unbiased-test_arr_pos.npy")
raw_data['test_data_neg_unbiased'] = np.load(folder_name + "unbiased-test_arr_neg.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos_biased = ImplicitDataset(raw_data['test_data_pos_biased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg_biased = ImplicitDataset(raw_data['test_data_neg_biased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_pos_unbiased = ImplicitDataset(raw_data['test_data_pos_unbiased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg_unbiased = ImplicitDataset(raw_data['test_data_neg_unbiased'], raw_data['max_user'], raw_data['max_item'])

In [31]:
#Code to avoid tf using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()

model = MODEL_CLASS(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=model, sampler=sampler,
                                     eval_save_prefix=OUTPUT_PATH + DATASET_NAME,
                                     item_serving_size=500)
auc_evaluator = AUC()

model.load(OUTPUT_PATH)

INFO:tensorflow:Restoring parameters from ./Output/cml-yahoo/


In [32]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos_biased, test_dataset_neg_biased])
model_trainer._sample_negatives(seed=10)

[Subsampling negative items]


## Biased Evaluation

In [33]:
model_trainer._eval_save_prefix = OUTPUT_PREFIX + "-test-pos-biased"
model_trainer._evaluate_partial(test_dataset_pos_biased)

model_trainer._eval_save_prefix = OUTPUT_PREFIX +  "-test-neg-biased"
model_trainer._evaluate_partial(test_dataset_neg_biased)

100%|██████████| 2296/2296 [00:25<00:00, 90.86it/s]


{'AUC': [0.5076271186440678,
  0.506271186440678,
  0.5027348993288591,
  0.5019463087248321,
  0.4898316498316499,
  0.4743728813559322,
  0.5206879194630873,
  0.5511317567567567,
  0.5093120805369128,
  0.513047138047138,
  0.5214915254237287,
  0.5127702702702702,
  0.4757286432160805,
  0.5069833333333333,
  0.5159548611111111,
  0.48922818791946315,
  0.5137959866220736,
  0.4623639455782313,
  0.4967281879194631,
  0.5175252525252525,
  0.5357859531772575,
  0.527277397260274,
  0.5184615384615384,
  0.44310402684563766,
  0.47668896321070237,
  0.49315140845070427,
  0.4975773195876289,
  0.49876254180602003,
  0.4879081632653061,
  0.52513468013468,
  0.4663087248322148,
  0.5041582491582492,
  0.47843749999999996,
  0.5076421404682275,
  0.4563590604026846,
  0.5035906040268456,
  0.514314381270903,
  0.4532993197278911,
  0.4963422818791946,
  0.5022972972972973,
  0.4911655405405405,
  0.44620689655172413,
  0.5423037542662117,
  0.4843771043771044,
  0.48109060402684567,
 

## Unbiased Evaluation

In [34]:
model_trainer._eval_save_prefix = OUTPUT_PREFIX + "-test-pos-unbiased"
model_trainer._evaluate_partial(test_dataset_pos_unbiased)

model_trainer._eval_save_prefix = OUTPUT_PREFIX +  "-test-neg-unbiased"
model_trainer._evaluate_partial(test_dataset_neg_unbiased)

100%|██████████| 2296/2296 [00:01<00:00, 1701.89it/s]


{'AUC': [0.5327777777777778,
  0.5640000000000001,
  0.5308333333333333,
  0.5066666666666666,
  0.4288888888888888,
  0.44214285714285717,
  0.32749999999999996,
  0.6371428571428572,
  0.35,
  0.48055555555555557,
  0.44375,
  0.4535714285714286,
  0.5471428571428572,
  0.50125,
  0.48,
  0.5177777777777777,
  0.5094444444444445,
  0.536875,
  0.5285714285714286,
  0.5299999999999999,
  0.460625,
  0.47,
  0.4027777777777778,
  0.49124999999999996,
  0.67875,
  0.30125,
  0.64,
  0.541875,
  0.3725,
  0.625,
  0.2735714285714285,
  0.5538888888888889,
  0.4783333333333333,
  0.49222222222222217,
  0.4511111111111111,
  0.62,
  0.608125,
  0.31666666666666665,
  0.5133333333333333,
  0.3807142857142857,
  0.5381250000000001,
  0.424375,
  0.580625,
  0.603,
  0.39222222222222225,
  0.48125000000000007,
  0.6128571428571429,
  0.33499999999999996,
  0.5572222222222222,
  0.44999999999999996,
  0.4677777777777778,
  0.5609999999999999,
  0.4491666666666667,
  0.42500000000000004,
  0.38

## Calculate Metrics

In [35]:
biased_results = dict()



# biased_results["STRATIFIED_15"] = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=30, partition=100)
biased_results["AOA"] = aoa(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", K=30)
biased_results["UB_15"] = eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=30)
biased_results["UB_2"] =  eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=30)
biased_results["UB_25"] =  eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=30)
biased_results["UB_3"] =  eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=30)

In [36]:
unbiased_results = dict()

# unbiased_results["STRATIFIED_15"] = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=1, partition=100)
unbiased_results["AOA"] = aoa(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", K=1)
unbiased_results["UB_15"] = eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=1)
unbiased_results["UB_2"] =  eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=1)
unbiased_results["UB_25"] =  eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=1)
unbiased_results["UB_3"] =  eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=1)

In [37]:
from tqdm.notebook import tqdm

In [38]:
# Compute biased and unbiased results with stratified for values of partition in (1,2*len(sorted_items))
# and store biased and unbiased results such that abs(biased_results[key]['auc'] - unbiased_results[key]['auc']) + abs(biased_results[key]['recall'] - unbiased_results[key]['recall']) is minimized
num_items = 932

unbiased_results["STRATIFIED_15"] = dict()
biased_results["STRATIFIED_15"] = dict()
best_partition=1

for p in tqdm(range(1, 2*num_items)):
    temp_unbiased = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=1, partition=p)
    temp_biased = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=30, partition=p)
    if not unbiased_results["STRATIFIED_15"]:
        unbiased_results["STRATIFIED_15"] = temp_unbiased
    if not biased_results["STRATIFIED_15"]:
        biased_results["STRATIFIED_15"] = temp_biased
    elif abs(temp_biased['auc'] - temp_unbiased['auc']) + abs(temp_unbiased['recall'] - temp_biased['recall']) < abs(biased_results["STRATIFIED_15"]['auc'] - unbiased_results["STRATIFIED_15"]['auc']) + abs(biased_results["STRATIFIED_15"]['recall'] - unbiased_results["STRATIFIED_15"]['recall']):
        biased_results["STRATIFIED_15"]['auc'] = temp_biased['auc']
        biased_results["STRATIFIED_15"]['recall'] = temp_biased['recall']
        unbiased_results["STRATIFIED_15"]['auc'] = temp_unbiased['auc']
        unbiased_results["STRATIFIED_15"]['recall'] = temp_unbiased['recall']
        best_partition = p


  0%|          | 0/1863 [00:00<?, ?it/s]

In [45]:
best_partition

60

In [39]:
unbiased_results["STRATIFIED_2"] = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=1, partition=best_partition)
biased_results["STRATIFIED_2"] = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=30, partition=best_partition)

unbiased_results["STRATIFIED_25"] = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=1, partition=best_partition)
biased_results["STRATIFIED_25"] = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=30, partition=best_partition)

unbiased_results["STRATIFIED_3"] = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=1, partition=best_partition)
biased_results["STRATIFIED_3"] = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=30, partition=best_partition)
    

In [40]:
key, value = random.choice(list(biased_results.items()))
rows = 2#len(list(value.keys()))
columns = 9#len(list(biased_results.items()))
results_array = np.zeros((rows,columns))

In [41]:
mae_results = dict()

list_biased_res = list(biased_results.keys())

for i in range(len(list_biased_res)):
    key = list_biased_res[i]

    for j in range(len(list(biased_results[key].keys()))):
        key_2 = list(biased_results[key].keys())[j]

        results_array[j][i] = abs(biased_results[key][key_2] - unbiased_results[key][key_2])


In [42]:
mae_df = pd.DataFrame(columns=list(biased_results.keys()), data=results_array)

In [43]:
metric_values = list(biased_results[list(biased_results.keys())[0]].keys())
mae_df.insert(0, "metric", metric_values)


# **RESULTS**

In [44]:
mae_df.head()

,metric,AOA,UB_15,UB_2,UB_25,UB_3,STRATIFIED_15,STRATIFIED_2,STRATIFIED_25,STRATIFIED_3
0,auc,0.152750,0.125714,0.122354,0.119810,0.117876,0.022406,0.014835,0.369804,0.208499
1,recall,0.378857,0.258728,0.247177,0.238659,0.232314,0.271626,0.398290,0.616611,2.394251
